In [161]:
import pathlib
import random
import pandas as pd
import numpy as np
import sys
import nltk

from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict

from sklearn.metrics import (
    f1_score, 
    accuracy_score,
    classification_report, 
)

ROOT_DIR = pathlib.Path().absolute()
DATA_DIR = ROOT_DIR / "data"
RANDOM_SEED = 42

## Загрузка и обзор данных

In [162]:
df_trends = pd.read_csv(DATA_DIR / "trends_description.csv")
df = pd.read_csv(DATA_DIR / "train.csv")
df_test = pd.read_csv(DATA_DIR / "test.csv")

In [163]:
df.head()

,Unnamed: 0,index,assessment,tags,text,trend_id_res0,trend_id_res1,trend_id_res2,trend_id_res3,trend_id_res4,...,trend_id_res40,trend_id_res41,trend_id_res42,trend_id_res43,trend_id_res44,trend_id_res45,trend_id_res46,trend_id_res47,trend_id_res48,trend_id_res49
0,0,5652,6.0,"{ASSORTMENT,PROMOTIONS,DELIVERY}","Маленький выбор товаров, хотелось бы ассортиме...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,18092,4.0,"{ASSORTMENT,PRICE,PRODUCTS_QUALITY,DELIVERY}",Быстро,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,13845,6.0,"{DELIVERY,PROMOTIONS,PRICE,ASSORTMENT,SUPPORT}",Доставка постоянно задерживается,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,25060,6.0,"{PRICE,PROMOTIONS,ASSORTMENT}",Наценка и ассортимент расстраивают,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1428,6.0,"{PRICE,PROMOTIONS}",Можно немного скинуть минимальную сумму заказа...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Обучение моделей

### Предобработка данных

In [164]:
df.head()

,Unnamed: 0,index,assessment,tags,text,trend_id_res0,trend_id_res1,trend_id_res2,trend_id_res3,trend_id_res4,...,trend_id_res40,trend_id_res41,trend_id_res42,trend_id_res43,trend_id_res44,trend_id_res45,trend_id_res46,trend_id_res47,trend_id_res48,trend_id_res49
0,0,5652,6.0,"{ASSORTMENT,PROMOTIONS,DELIVERY}","Маленький выбор товаров, хотелось бы ассортиме...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,18092,4.0,"{ASSORTMENT,PRICE,PRODUCTS_QUALITY,DELIVERY}",Быстро,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,13845,6.0,"{DELIVERY,PROMOTIONS,PRICE,ASSORTMENT,SUPPORT}",Доставка постоянно задерживается,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,25060,6.0,"{PRICE,PROMOTIONS,ASSORTMENT}",Наценка и ассортимент расстраивают,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1428,6.0,"{PRICE,PROMOTIONS}",Можно немного скинуть минимальную сумму заказа...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [165]:
X, y = df[["text"]], df[[f"trend_id_res{i}" for i in range(50)]]
X = X.astype("str").copy()
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size= 0.8, random_state = 42)
print(f"X_train.shape is {X_train.shape}")
print(f"y_train.shape is {y_train.shape}")
print(f"X_val.shape is {X_val.shape}")
print(f"y_val.shape is {y_val.shape}")
X_test = df_test[["text"]]
print(f"X_test.shape is {X_test.shape}")
trend_info = pd.read_csv(DATA_DIR / "trends_description.csv")


X_train.shape is (3698, 1)
y_train.shape is (3698, 50)
X_val.shape is (925, 1)
y_val.shape is (925, 50)
X_test.shape is (9015, 1)


In [93]:

categories = []
for i in trend_info['trend']:
    categories.append(i)
categories

['Долгая доставка',
 'Доставка стала долгой',
 'Время доставки не соответствует заявленому',
 'Регулярные опоздания',
 'Не отследить реальное время доставки',
 'Курьер на карте',
 'Нет доставки по адресу',
 'Не предупреждаем об удалении товара',
 'Высокая минимальная сумма заказа',
 'Сумма заказа меняется во время набора корзины',
 'Минимальная сумма заказа',
 'Товары с подходящим сроком годности',
 'Высокие цены',
 'Не довезли товар',
 'Товар испорчен во время доставки',
 'Просроченные товары',
 'Замечания по работе курьеров',
 'Не читаем комментарии',
 'Спасибо',
 'Нет смысла',
 'Всё нормально',
 'Всё плохо',
 'Скидки для постоянных клиентов',
 'Больше акций/скидок',
 'Скидка/промокод распространяется не на все товары',
 'Непонятно как работает скидка',
 'Не сработала скидка/акция/промокод',
 'Качество товаров',
 'Маленький ассортимент',
 'Нет в наличии товара',
 'Качество поддержки',
 'Замечания по работе сборщика',
 'Отменили заказ',
 'Знание русского языка',
 'Привезли чужой заказ

In [94]:
X_train

,text
1538,"Ну, за [NUM]ч. и [NUM] мин. мне ещё никогда не..."
2991,Доставка всегда осуществляется значительно дол...
2812,Задержка доставки
4515,"Отличный сервис, только бы ассортимент расшири..."
4531,"Поддержка говно, курьеры опаздывают минут на [..."
...,...
4426,"+ быстро. - иногда сумма заказа очень велика, ..."
466,🦉
3092,До самоката я тратил меньше денег в день
3772,О вас редко думаю. Напрягает СТМ. Когда непоня...


In [98]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3), lowercase=False,  min_df=0.02,max_features=1000)
vectors=vectorizer.fit_transform(X_train['text'])
vectors.shape

(3698, 99)

In [100]:
vectorizer.get_feature_names_out()


array(['NUM', 'NUM NUM', 'NUM NUM минут', 'NUM мин', 'NUM минут', 'Все',
       'Долго', 'Доставка', 'Курьеры', 'Не', 'Очень', 'Очень долгая',
       'Очень долгая доставка', 'Очень долго', 'Раньше', 'Цены', 'Часто',
       'ассортимент', 'больше', 'будет', 'бы', 'было', 'вас', 'времени',
       'время', 'время доставки', 'все', 'всегда', 'вы', 'выше',
       'годности', 'до', 'долгая', 'долгая доставка', 'долго', 'дольше',
       'доставка', 'доставки', 'доставку', 'доставляют', 'если', 'ждать',
       'же', 'за', 'за NUM', 'за NUM минут', 'заказ', 'заказа', 'заказы',
       'из', 'или', 'иногда', 'как', 'качество', 'когда', 'курьер',
       'курьеры', 'лучше', 'мин', 'минут', 'можно', 'на', 'не', 'нет',
       'но', 'нравится', 'нужно', 'овощи', 'от', 'от NUM', 'очень', 'по',
       'поддержка', 'последнее', 'последнее время', 'постоянно', 'при',
       'привозят', 'продуктов', 'продукты', 'просто', 'раз', 'сейчас',
       'сервис', 'так', 'то', 'товаров', 'товары', 'только', 'уже',


In [78]:
dense_vectors = vectors.todense()
dense_vectors.shape

(3698, 99)

In [195]:
from sklearn.svm import SVC
preprocessor = ColumnTransformer(
    [
        ("vetorizer", TfidfVectorizer(analyzer="char_wb", ngram_range = (1,3), lowercase=False,max_features=1000), "text")
    ],                         
    remainder = "passthrough"
)

pipeline_multiout = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("clf", MultiOutputClassifier(SVC(kernel='rbf', gamma=0.1, C=2))),
    ]
)
display(pipeline_multiout)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('vetorizer',
                                                  TfidfVectorizer(analyzer='char_wb',
                                                                  lowercase=False,
                                                                  max_features=1000,
                                                                  ngram_range=(1,
                                                                               3)),
                                                  'text')])),
                ('clf', MultiOutputClassifier(estimator=SVC(C=2, gamma=0.1)))])

In [196]:
cross_valid = cross_validate(pipeline_multiout, 
                             X_train, y_train, 
                             cv = 5, scoring = ["accuracy"], n_jobs = -1)
print("test_accuracy:", cross_valid["test_accuracy"].mean())

test_accuracy: 0.23363639688402885


In [197]:
y_pred = cross_val_predict(pipeline_multiout, X_train, y_train, cv = 2)

In [198]:
# Посмотрим на различные метрики
print(classification_report(y_train, y_pred, zero_division = 0))

              precision    recall  f1-score   support

           0       0.88      0.41      0.56       661
           1       0.50      0.01      0.01       270
           2       0.76      0.31      0.45       486
           3       0.90      0.19      0.31       289
           4       0.00      0.00      0.00       108
           5       0.00      0.00      0.00        44
           6       0.00      0.00      0.00        16
           7       0.00      0.00      0.00        27
           8       0.00      0.00      0.00       109
           9       0.00      0.00      0.00         9
          10       0.00      0.00      0.00        76
          11       0.00      0.00      0.00        87
          12       0.95      0.51      0.67       491
          13       0.00      0.00      0.00        29
          14       0.00      0.00      0.00        62
          15       0.00      0.00      0.00        66
          16       0.00      0.00      0.00       166
          17       0.00    

In [199]:
# Посмотрим на целевую метрику
accuracy_score(y_train, y_pred)

0.19226608977825851

###  Тренировка окончательной модели

In [200]:
pipeline_multiout.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('vetorizer',
                                                  TfidfVectorizer(analyzer='char_wb',
                                                                  lowercase=False,
                                                                  max_features=1000,
                                                                  ngram_range=(1,
                                                                               3)),
                                                  'text')])),
                ('clf', MultiOutputClassifier(estimator=SVC(C=2, gamma=0.1)))])

##  Предсказание и загрузка решения

In [201]:
pred_test = pipeline_multiout.predict(df_test[["text"]].astype("str"))

In [202]:
res = pd.DataFrame(np.hstack([df_test["index"].values.reshape(df_test.shape[0], 1), pred_test]),
                  columns = ["index"]+[f"{i}" for i in range(50)])

In [203]:
res.head()

,index,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,3135,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4655,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,22118,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,23511,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,45,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [204]:

res["target"] = res.drop('index',axis=1).apply(lambda r: r.index[r.ne(0)].to_list(), axis=1)


res

,index,0,1,2,3,4,5,6,7,8,...,41,42,43,44,45,46,47,48,49,target
0,3135,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[]
1,4655,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[12]
2,22118,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[2]
3,23511,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[0]
4,45,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9010,3523,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[]
9011,24925,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[]
9012,6327,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[]
9013,530,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[]


In [205]:
f=open('submission.csv', 'w')
f.write("index,target")
f.write("\n")
for i in range(len(res)):
    
    f.write(str(res['index'][i]))
    f.write(',')
    for x in res['target'][i]:
        f.write(str(x))
        f.write(' ')
    f.write("\n")

f.close()

In [18]:
res.iloc[:, 1:].sum()

0                                                       750
1                                                       174
2                                                       706
3                                                       222
4                                                         0
5                                                         0
6                                                         0
7                                                         0
8                                                        23
9                                                         0
10                                                        0
11                                                       30
12                                                      743
13                                                        0
14                                                        0
15                                                        5
16                                      

In [19]:
res["0"].value_counts()

0
0    8265
1     750
Name: count, dtype: int64